# Load all the pig dicom data and convert them to nifti

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
# !pip install loguru pydicom scipy scikit-image

# !pip install git+https://github.com/mjirik/io3d --upgrade --force-reinstall
# !pip install git+https://github.com/mjirik/imma --upgrade --force-reinstall

# !pip install git+https://github.com/mjirik/io3d
# !pip install git+https://github.com/mjirik/imma



In [51]:
import io3d
from pathlib import Path
from loguru import logger
from pprint import pprint, pformat
import pandas as pd
import re
import numpy as np
import json
import tqdm

logger.enable("io3d")
force = True
# force = False

recreate_meta = False
recreate_meta = True
# base_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_dicom")
dirname = "Prasata DC"
base_path = Path(r"~/mnt/nas-bmc3_ct/" + dirname).expanduser()
fn_prefix = dirname.replace(" ", '_') + "-"
output_dir_part = dirname.replace(" ", '_')
# base_path = Path(r"~/Downloads/_temp/").expanduser()  #: used when the zip files are downloaded manually
# raw_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_nii_transposed")
# transposed_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_nii_transposed")
# output_path = Path(r"H:\biomedical\orig\pilsen_pigs")
output_path= Path(r"~/mnt/nas-bmc3_ct/pilsen_pigs_all/transplantation_nii_transposed").expanduser() / output_dir_part

assert base_path.exists()

# Convert all to nii

In [52]:

def touch_file(pth:Path):
    try:
        with open(pth, "rb") as f:
            # read just some part of the file
            f.read(1)
    except Exception as e:
        import traceback
        print(f"Error in touching file {pth}: {e}")
        traceback.print_exc()

In [53]:
from typing import Union

def get_projection(
        datap:io3d.image.DataPlus, axis:Union[int,str], method:str="max"
):
    """Get projection of 3D data to 2D."""
    if isinstance(axis, str):
        dict_axis = {"axial": 0, "coronal": 1, "sagittal": 2}
        if axis in dict_axis:
            axis = dict_axis[axis]
        else:
            raise ValueError(f"Unknown axis {axis}, use one of {list(dict_axis.keys())} or 0, 1, 2")

    data3d = datap.data3d
    axcodes = datap.orientation_axcodes
    data3d = io3d.image.transform_orientation(data3d, axcodes, "SPL")
    if method == "max":
        data2d = data3d.max(axis=axis)
    elif method == "mean":
        data2d = data3d.mean(axis=axis)
    else:
        raise ValueError(f"Unknown method {method}")
    return data2d



In [54]:
# data2d = get_projection(datap, 0, "max")
# from matplotlib import pyplot as plt
# plt.style.use('classic')
# plt.imshow(data2d, cmap="gray_r")
# plt.colorbar()
# plt.style.available


In [55]:
import tqdm

In [56]:
# List all directories under base_path (recursively)
leaf_dirs = []
for d in tqdm.tqdm(list(base_path.glob("**/"))):
    if d.is_dir():
        # Get direct children
        children = list(d.iterdir())
        # Identify if there are any subdirectories
        has_subdir = any(child.is_dir() for child in children)
        # Identify if there is at least one file in the directory
        has_file = any(child.is_file() for child in children)
        # A leaf directory has no subdirectories but does contain files
        if not has_subdir and has_file:
            leaf_dirs.append(d)

leaf_dirs

100%|██████████| 1122/1122 [01:07<00:00, 16.68it/s]


[PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500000'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500001'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500002'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500003'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500004'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500005'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500006'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500007'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500008'),
 PosixPath('/home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500009'),
 PosixPath('/home/mjirik/mnt/n

In [57]:
str(leaf_dirs[0].relative_to(base_path))

'2022/C11 24.5.2022/DICOM/22052411/29500000'

In [58]:
io3d.read(sorted(leaf_dirs)[0], orientation_axcodes="IPL")

2025-04-09 16:19:07.500 | DEBUG    | io3d.datareader:__ReadFromDirectory:261 - Dir - Image sequence
2025-04-09 16:19:07.500 | DEBUG    | io3d.datareader:__ReadFromDirectory:263 - Getting list of readable files...
2025-04-09 16:19:53.515 | DEBUG    | io3d.datareader:__ReadFromDirectory:284 - Getting numpy array from image data...


{'series_number': 0,
 'datadir': '/home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500000',
 'voxelsize_mm': [0.4, 0.717953125, 0.717953125],
 'orientation_axcodes': 'IPL',
 'data3d': array([[[-8192, -8192, -8192, ..., -1012, -8192, -8192],
         [-8192, -8192, -8192, ..., -1009, -1005, -8192],
         [-8192, -8192, -8192, ..., -1018, -1012, -1022],
         ...,
         [-8192, -8192, -8192, ...,  -988, -1002, -1019],
         [-8192, -8192, -8192, ...,  -999, -1014, -8192],
         [-8192, -8192, -8192, ..., -1013, -8192, -8192]],
 
        [[-8192, -8192, -8192, ..., -1023, -8192, -8192],
         [-8192, -8192, -8192, ..., -1020, -1010, -8192],
         [-8192, -8192, -8192, ..., -1013, -1026, -1026],
         ...,
         [-8192, -8192, -8192, ...,  -992,  -999, -1013],
         [-8192, -8192, -8192, ..., -1000, -1004, -8192],
         [-8192, -8192, -8192, ..., -1004, -8192, -8192]],
 
        [[-8192, -8192, -8192, ..., -1018, -8192, -8192],
   

In [60]:
# fnlist = list(base_path.glob("*Tx0*D_V*"))
# find a leaf directory in base_path which contains some file



# fnlist = sorted(list(base_path.glob("*Tx0*D_A*")) + list(base_path.glob("*Tx0*D_V*")))[::-1]
fnlist = sorted(leaf_dirs)


print(f"Number of files: {len(fnlist)}")
from joblib import Parallel, delayed
import tqdm
#
for fn in tqdm.tqdm(fnlist):

    # take all files in parallel in the fn directory and open them and close, to intiate cloud syncronization
    # import Parallel, delayed
    # i want to have there the parent directory
    # fn_name = str(fn.relative_to(base_path)).replace("\\", "-").replace("/", "-")
    fn_name = (fn_prefix + str(fn.relative_to(base_path))).replace("\\", "-").replace("/", "-").replace(" ", "_")

    # logger.info(fn)
    fn_in = fn
    # fn_out = raw_path / fn.name / f"{fn.name}.mhd"
    fn_out = output_path / fn_name / f"{fn_name}.nii.gz"

    fn_out.parent.mkdir(parents=True, exist_ok=True)
    fn_meta = fn_out.parent / "meta.json"
    if force or (not fn_out.exists()):

        try:
            fn_fns = sorted(list(fn.glob("*")))
            tqdm.tqdm.write(f"Reading {len(fn_fns)} files in {fn_in} and writing {fn_out}")
            # tqdm.tqdm.write(f"Number of files in the directory: {len(fn_fns)}")
            # Parallel(n_jobs=2)(delayed(touch_file)(fn) for fn in tqdm.tqdm(fn_fns, desc="touching files"))

            axcodes = "IPL"
            # logger.debug(f"Reading {fn_in} with axcodes={axcodes}")
            datap = io3d.read(fn_in, series_number="first", orientation_axcodes=axcodes)
            # logger.debug(datap.keys())
            io3d.write(datap, fn_out)
            # logger.debug("writing done, creating projections")
            for axis in ["axial", "coronal", "sagittal"]:
                data2d = get_projection(datap, axis, "max")
                import skimage.io
                # change intensity to range 0..1

                data2d = (255 * (data2d - np.min(data2d)).astype(float) / (np.max(data2d) - np.min(data2d))).astype(np.uint8)
                skimage.io.imsave(fn_out.parent / f"{fn_out.stem}_{axis}.jpg", data2d)
            # logger.debug("projections done")
        except Exception as e:
            import traceback
            logger.error(f"Error in reading {fn_in}: {e}")
            traceback.print_exc()
            # logger.debug(f"shape={datap.data3d.shape}, {datap.orientation_axcodes}")
        # with open(fn_meta, "w") as f:
        #     json.dump(dict(row), f)





Number of files: 871


  0%|          | 0/871 [00:00<?, ?it/s]2025-04-09 16:21:37.808 | DEBUG    | io3d.datareader:__ReadFromDirectory:261 - Dir - Image sequence
2025-04-09 16:21:37.809 | DEBUG    | io3d.datareader:__ReadFromDirectory:263 - Getting list of readable files...


Reading 500 files in /home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500000 and writing /home/mjirik/mnt/nas-bmc3_ct/pilsen_pigs_all/transplantation_nii_transposed/Prasata_C/Prasata_C-2022-C11_24.5.2022-DICOM-22052411-29500000/Prasata_C-2022-C11_24.5.2022-DICOM-22052411-29500000.nii.gz


2025-04-09 16:22:03.134 | DEBUG    | io3d.datareader:__ReadFromDirectory:284 - Getting numpy array from image data...
  0%|          | 1/871 [00:32<7:51:38, 32.53s/it]2025-04-09 16:22:10.776 | DEBUG    | io3d.datareader:__ReadFromDirectory:261 - Dir - Image sequence
2025-04-09 16:22:10.778 | DEBUG    | io3d.datareader:__ReadFromDirectory:263 - Getting list of readable files...


Reading 500 files in /home/mjirik/mnt/nas-bmc3_ct/Prasata C/2022/C11 24.5.2022/DICOM/22052411/29500001 and writing /home/mjirik/mnt/nas-bmc3_ct/pilsen_pigs_all/transplantation_nii_transposed/Prasata_C/Prasata_C-2022-C11_24.5.2022-DICOM-22052411-29500001/Prasata_C-2022-C11_24.5.2022-DICOM-22052411-29500001.nii.gz


...

ImageSeriesReader (0x16691a50): Non uniform sampling or missing slices detected,  maximum nonuniformity:167.129

2025-04-10 01:37:42.112 | DEBUG    | io3d.datareader:__ReadFromDirectory:284 - Getting numpy array from image data...
/tmp/ipykernel_321757/3975504077.py:48: RuntimeWarning: overflow encountered in scalar subtract
  data2d = (255 * (data2d - np.min(data2d)).astype(float) / (np.max(data2d) - np.min(data2d))).astype(np.uint8)
100%|█████████▉| 869/871 [9:16:13<01:11, 35.56s/it]2025-04-10 01:37:51.036 | DEBUG    | io3d.datareader:__ReadFromDirectory:261 - Dir - Image sequence
2025-04-10 01:37:51.036 | DEBUG    | io3d.datareader:__ReadFromDirectory:263 - Getting list of readable files...


Reading 500 files in /home/mjirik/mnt/nas-bmc3_ct/Prasata C/2025/C37 29.3.2025/DICOM/25032908/17450005 and writing /home/mjirik/mnt/nas-bmc3_ct/pilsen_pigs_all/transplantation_nii_transposed/Prasata_C/Prasata_C-2025-C37_29.3.2025-DICOM-25032908-17450005/Prasata_C-2025-C37_29.3.2025-DICOM-25032908-17450005.nii.gz


2025-04-10 01:38:15.789 | DEBUG    | io3d.datareader:__ReadFromDirectory:284 - Getting numpy array from image data...
100%|█████████▉| 870/871 [9:16:46<00:34, 34.81s/it]2025-04-10 01:38:23.960 | DEBUG    | io3d.datareader:__ReadFromDirectory:261 - Dir - Image sequence
2025-04-10 01:38:23.961 | DEBUG    | io3d.datareader:__ReadFromDirectory:263 - Getting list of readable files...


Reading 201 files in /home/mjirik/mnt/nas-bmc3_ct/Prasata C/2025/C37 29.3.2025/DICOM/25032908/17450006 and writing /home/mjirik/mnt/nas-bmc3_ct/pilsen_pigs_all/transplantation_nii_transposed/Prasata_C/Prasata_C-2025-C37_29.3.2025-DICOM-25032908-17450006/Prasata_C-2025-C37_29.3.2025-DICOM-25032908-17450006.nii.gz


2025-04-10 01:38:28.843 | WARNING  | io3d.datareader:__ReadFromDirectory:275 - Cant load file: 36508186
2025-04-10 01:38:28.843 | WARNING  | io3d.datareader:__ReadFromDirectory:277 - Exception thrown in SimpleITK ImageFileReader_Execute: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:99:
sitk::ERROR: Unable to determine ImageIO reader for "/home/mjirik/mnt/nas-bmc3_ct/Prasata C/2025/C37 29.3.2025/DICOM/25032908/17450006/36508186"
2025-04-10 01:38:31.519 | DEBUG    | io3d.datareader:__ReadFromDirectory:284 - Getting numpy array from image data...
100%|██████████| 871/871 [9:16:56<00:00, 38.37s/it]


In [36]:
# import tqdm
# import time
# for i in tqdm.tqdm(range(100)):
#     #wait 0.1s
#     time.sleep(0.1)
    

In [37]:
metafn = base_path / "meta.csv"

if metafn.exists():
    df = pd.read_csv(metafn)
else:
    df = pd.DataFrame()


In [38]:
def create_meta(base_path, metafn):
    data = {
        "dirname": [],
        "dataset_type": [],
        "id": [],
    }
    i_train = 0
    i_test = 0
    i_val = 0
    fnlist = base_path.glob("*Tx0*D_V*")
    logger.info(pformat(list(map(str, list(fnlist)))))
    fnlist = base_path.glob("*Tx0*D_V*")
    for fn in fnlist:
        ia = int(re.findall(r"([0-9]+)", fn.name)[0])
        ii = None
        tp = None
        if (ia % 4) == 0:
            i_test += 1
            ii = i_test
            tp = "test"
        elif (ia % 2) == 0:
            i_val += 1
            ii = i_val
            tp = "val"
        else:
            i_train += 1
            ii = i_train
            tp = "train"

        data["dirname"].append(fn.name)
        data["dataset_type"].append(tp)
        data["id"].append(ii)

    df = pd.DataFrame(data)
    df.to_csv(metafn, index=None)



In [39]:
if recreate_meta:
    create_meta(base_path, metafn)


2025-04-03 21:08:24.784 | INFO     | __main__:create_meta:11 - []


In [40]:

logger.debug(metafn.exists())
df = pd.read_csv(metafn)
print(df)


2025-04-03 21:08:24.827 | DEBUG    | __main__:<module>:1 - True


Empty DataFrame
Columns: [dirname, dataset_type, id]
Index: []


In [41]:
# fnlist = base_path.glob("*debug*")
# fnlist = base_path.glob("*Tx041D_V*")


# for i, row in tqdm.tqdm(df.iterrows()):
#     fn_in = base_path / row["dirname"]
#     fn_out = (
#             output_path
#             / row["dataset_type"]
#             / f"PP_{row['id']:04}"
#             / "PATIENT_DICOM"
#             / f"PP_{row['id']:04}.mhd"
#     )
#     fn_meta = fn_out.parent / "meta.json"
#     logger.debug(fn_in)
#     logger.debug(fn_out)
#
#     fn_out.parent.mkdir(parents=True, exist_ok=True)
#     if (
#             not fn_meta.exists()
#     ):  # we do not need to read the data again if everything is done. We are checkin:w
#
#         tqdm.tqdm.write(f"Reading {fn_in} and writing {fn_out}")
#         datap = io3d.read(fn_in)
#         io3d.write(datap, fn_out)
#         with open(fn_meta, "w") as f:
#             json.dump(dict(row), f)
